In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from optimus import *

from pyspark.sql.session import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, IntegerType, ArrayType

sc = SparkSession.builder.getOrCreate()

In [3]:
# Create optimus
op = Optimus(sc)

Using a created Spark Session...
Done.


## Create dataframe
### Spark

This is ugly:

```
val someData = Seq(
  Row(8, "bat"),
  Row(64, "mouse"),
  Row(-27, "horse")
)

val someSchema = List(
  StructField("number", IntegerType, true),
  StructField("word", StringType, true)
)

val someDF = spark.createDataFrame(
  spark.sparkContext.parallelize(someData),
  StructType(someSchema)
)```

In [188]:
# Thanks Mr Powers
df = op.create.df([
                ("  I like     fish  ", 1, "dog", "&^%$#housé", "cat-car", "a","1", "20150510", "3"),
                ("    zombies", 2, "cat", "tv", "dog-tv", "b","2", "20160510", "3"),
                ("simpsons   cat lady", 2, "frog", "table","eagle-tv-plus","1","3", "20170510", "4"),
                (None, 3, "eagle", "glass", "lion-pc", "c","4", "20180510", "5"),
    
            ],
            [
                ("words", "str", True),
                ("num", "int", True),
                ("animals", "str", True),
                ("thing", StringType(), True),
                ("two strings", StringType(), True),
                ("filter", StringType(), True),
                ("num 2", "string", True),
                ("date", "string", True),
                ("num 3", "string", True)
                
            ])

df.show()

+-------------------+---+-------+----------+-------------+------+-----+--------+-----+
|              words|num|animals|     thing|  two strings|filter|num 2|    date|num 3|
+-------------------+---+-------+----------+-------------+------+-----+--------+-----+
|  I like     fish  |  1|    dog|&^%$#housé|      cat-car|     a|    1|20150510|    3|
|            zombies|  2|    cat|        tv|       dog-tv|     b|    2|20160510|    3|
|simpsons   cat lady|  2|   frog|     table|eagle-tv-plus|     1|    3|20170510|    4|
|               null|  3|  eagle|     glass|      lion-pc|     c|    4|20180510|    5|
+-------------------+---+-------+----------+-------------+------+-----+--------+-----+



In [179]:
# https://stackoverflow.com/questions/47906697/pyspark-pass-list-as-parameter-to-udf

import unicodedata

def return_age_bracket(attr):
    print(type(attr))
    cell_str = attr
    print(cell_str)
    # first, normalize strings:
    nfkd_str = unicodedata.normalize('NFKD', cell_str)
    # Keep chars that has no other char combined (i.e. accents chars)
    with_out_accents = u"".join([c for c in nfkd_str if not unicodedata.combining(c)])
    return with_out_accents
    print(attr)

udf_function = udf(return_age_bracket)


params= ["thing"]
for p in params:
    df.withColumn("thing", udf_function(df[p])).show()
    

+-------------------+---+-------+-----+-------------+------+-----+--------+-----+
|              words|num|animals|thing|  two strings|filter|num 2|    date|num 3|
+-------------------+---+-------+-----+-------------+------+-----+--------+-----+
|  I like     fish  |  1|    dog|house|      cat-car|     a|    1|20150510|    3|
|            zombies|  2|    cat|   tv|       dog-tv|     b|    2|20160510|    3|
|simpsons   cat lady|  2|   frog|table|eagle-tv-plus|     1|    3|20170510|    4|
|               null|  3|  eagle|glass|      lion-pc|     c|    4|20180510|    5|
+-------------------+---+-------+-----+-------------+------+-----+--------+-----+

